In [1]:
import pandas as pd
import numpy as np
import math
import warnings
import model_metrics_helper
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.float_format = '{:,.4f}'.format

### Load Test Data

In [2]:
test_scores_df = pd.read_csv('/gpfs/data/paulab/sepsis_floor_prediction/sepsis_real_time_prediction/model_results/test_set_results_20220701_221434.csv')
test_scores_df.rename(columns={'SepsisTrueLabel':'SepsisLabel'}, inplace=True)
test_scores_df.iloc[:, 1:].head(5)

,LOS,PredictedProbability,SepsisLabel,t_timezero,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,SepsisLabel_0.6,SepsisLabel_0.7
0,0,0.0081,0,NaN,0,0,0,0,0,0,0,0,0
1,3,0.0052,0,NaN,0,0,0,0,0,0,0,0,0
2,6,0.0052,0,NaN,0,0,0,0,0,0,0,0,0
3,9,0.0043,0,NaN,0,0,0,0,0,0,0,0,0
4,12,0.0064,0,NaN,0,0,0,0,0,0,0,0,0


### Run SIRS and MEWS for the test data to compare as baselines with ML results at different thresholds

In [3]:
main_df = pd.read_csv("/gpfs/data/paulab/bvg228/sepsis_real_time_prediction/data/NYU_6hr_preprocessed_48hr_main_df_20220526_100348.csv")
main_df = main_df[main_df['ID'].isin(test_scores_df['ID'])]
main_df.rename(columns={'WHITE BLOOD CELL COUNT':'WBC', 'Systolic_BP':'SBP', 'GCS_Score':'gcs_total_score'}, inplace=True)
test_scores_df = test_scores_df.merge(main_df[['ID', 'LOS', 'rel_time']], on=['ID', 'LOS'], how='left')
test_scores_df['rel_time'] = test_scores_df['rel_time'].astype(int)
admission_time = main_df[['ID', 'AdmissionInstant']].drop_duplicates(keep='first')
test_scores_df = test_scores_df.merge(admission_time, on=['ID'], how='left') 
test_scores_df['AlertTime'] = test_scores_df['LOS'] + 1


# SIRS score
SIRS = model_metrics_helper.SIRS(main_df)
SIRS['rel_time'] = SIRS['rel_time'].astype(int)
# MEWS score
MEWS = model_metrics_helper.MEWS(main_df)
MEWS['rel_time'] = MEWS['rel_time'].astype(int)

# merge with prediction results
test_scores_df = test_scores_df.merge(SIRS, on = ['ID', 'rel_time'], how = "left")
test_scores_df = test_scores_df.merge(MEWS, on = ['ID', 'rel_time'], how = "left")

test_scores_df["SIRS"] = test_scores_df["SIRS"] >= 2
test_scores_df["MEWS"] = test_scores_df["MEWS"] >= 5

test_scores_df['SIRS'] = test_scores_df['SIRS'].astype(int)
test_scores_df['MEWS'] = test_scores_df['MEWS'].astype(int)
test_scores_df.rename(columns={'SIRS':'SepsisLabel_SIRS', 'MEWS':'SepsisLabel_MEWS'}, inplace=True)
test_scores_df.iloc[:, 1:].head(5)

,LOS,PredictedProbability,SepsisLabel,t_timezero,SepsisLabel_0.1,SepsisLabel_0.15,SepsisLabel_0.2,SepsisLabel_0.25,SepsisLabel_0.3,SepsisLabel_0.4,SepsisLabel_0.5,SepsisLabel_0.6,SepsisLabel_0.7,rel_time,AdmissionInstant,AlertTime,SepsisLabel_SIRS,SepsisLabel_MEWS
0,0,0.0081,0,NaN,0,0,0,0,0,0,0,0,0,1,2015-08-13 21:17:00,1,0,0
1,3,0.0052,0,NaN,0,0,0,0,0,0,0,0,0,4,2015-08-13 21:17:00,4,0,0
2,6,0.0052,0,NaN,0,0,0,0,0,0,0,0,0,7,2015-08-13 21:17:00,7,0,0
3,9,0.0043,0,NaN,0,0,0,0,0,0,0,0,0,10,2015-08-13 21:17:00,10,0,0
4,12,0.0064,0,NaN,0,0,0,0,0,0,0,0,0,13,2015-08-13 21:17:00,13,0,0


### Run threshold metrics

In [6]:
def threshold_metrics(df, threshold):
    actual = df["SepsisLabel"]
    predicted = df[f'SepsisLabel_{threshold}']
    tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
    
    acc = (tp + tn)/(tp + tn + fn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    sens = tp/ (tp + fn)
    spec = tn/ (tn + fp)
    f1 = (2*tp) / (2*tp + fp + fn)
    
    return [int(tp), int(fn), int(fp), int(tn), acc, ppv, npv, sens, spec, f1]

def threshold_first_alert(test_scores_df, threshold):
    df = test_scores_df[test_scores_df[f'SepsisLabel_{threshold}'] == 1]
    n_total_possible_sepsis_alerts = df.shape[0]
    n_total_possible_sepsis_alerts_day_yr = n_total_possible_sepsis_alerts / 365.
    df.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    df['LeadTimeValue'] = -(df['t_timezero'] - df['AlertTime'])
    df['abs_time'] = pd.to_datetime(test_scores_df['AdmissionInstant']) + pd.to_timedelta(test_scores_df['AlertTime'], unit='h')
    df['time_to_first_alert'] = (pd.to_datetime(df['abs_time']) - pd.to_datetime(df['AdmissionInstant'])).dt.total_seconds() / 3600.
    
    n_total_first_sepsis_alert = df.shape[0]
    n_total_first_sepsis_alert_day_yr = n_total_first_sepsis_alert / 365.
    df.dropna(subset=['t_timezero'], inplace=True) #there's no false positive. because dropped NA.
    n_tp_first_sepsis_alert = df.shape[0]
    n_early_first_sepsis_alert = df[df['LeadTimeValue'] < 0].shape[0]
    median_first_alert_time = df.groupby(f'SepsisLabel_{threshold}')['LeadTimeValue'].median().values[0]
    mean_first_alert_time_from_admission = df.groupby(f'SepsisLabel_{threshold}')['time_to_first_alert'].mean().values[0]
    return [n_total_possible_sepsis_alerts, n_total_possible_sepsis_alerts_day_yr,  n_total_first_sepsis_alert, n_total_first_sepsis_alert_day_yr, n_tp_first_sepsis_alert, n_early_first_sepsis_alert, median_first_alert_time, mean_first_alert_time_from_admission]

def final_threshold_metric_df(test_scores_df, thresholds):
    threshold_df = pd.DataFrame()
    threshold_df.index = ['tp', 'fn', 'fp', 'tn', 'acc', 'ppv', 'npv', 'sens', 'spec', 'F1']
    for threshold in thresholds:
        threshold_df[f'SepsisLabel_{threshold}'] = threshold_metrics(test_scores_df, threshold)
    threshold_df = threshold_df.T.reset_index().rename(columns={'index':'Threshold'})
    cols_to_int = ['tp', 'fn', 'fp', 'tn']
    for col in cols_to_int:
        threshold_df[col] = threshold_df[col].astype(int)
        
    lead_time_df = pd.DataFrame()
    lead_time_df.index = ['N_Total_Possible_Sepsis_Alerts', 'N_Total_Possible_Sepsis_Alerts_Per_Day', 'N_Total_First_Sepsis_Alert', 'N_Total_First_Sepsis_Alert_Per_Day','N_TP_First_Sepsis_Alert', 'N_Early_First_Sepsis_Alert', 'Median_First_Alert_Time_From_TimeZero', 'Avg_First_Alert_Time_From_Admission']
    for threshold in thresholds:
        lead_time_df[f'SepsisLabel_{threshold}'] = threshold_first_alert(test_scores_df, threshold)
    lead_time_df = lead_time_df.T.reset_index().rename(columns={'index':'Threshold'})
    cols_to_int = ['N_Total_Possible_Sepsis_Alerts', 'N_Total_Possible_Sepsis_Alerts_Per_Day', 'N_Total_First_Sepsis_Alert', 'N_Total_First_Sepsis_Alert_Per_Day','N_TP_First_Sepsis_Alert', 'N_Early_First_Sepsis_Alert']
    for col in cols_to_int:
        lead_time_df[col] = lead_time_df[col].astype(int)
             
    final_df = threshold_df.merge(lead_time_df, on=['Threshold'], how='left')
    final_df['Threshold'] = final_df['Threshold'].apply(lambda x: 'ML_' + x.split('_')[1] if x not in ['SepsisLabel_SIRS', 'SepsisLabel_MEWS'] else x.split('_')[1])
    
    return final_df
    


In [7]:
thresholds=[0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.09,  0.10, 0.15, 0.20, 0.25, 0.3, 0.4,0.5, 'SIRS', 'MEWS']

# test scores with full test set
final_df_w_poa = final_threshold_metric_df(test_scores_df, thresholds)

#test scores with t_timezero.isna() OR t_timezero > 1
test_scores_df_no_poa = test_scores_df[(test_scores_df['t_timezero'] > 1) | (test_scores_df['t_timezero'].isna())]
final_df_no_poa = final_threshold_metric_df(test_scores_df_no_poa, thresholds)

#combine both dataframes
final_full_df = pd.concat([final_df_w_poa, final_df_no_poa], keys=['Inclusion Criteria: All Test Set', 'Inclusion Criteria: Encounters with t_timezero.isna() OR t_timezero > 1'], axis=1)
#final_full_df.to_excel('/gpfs/data/paulab/sepsis_floor_prediction/sepsis_real_time_prediction/model_results/threshold_metrics_20220627_172236.xlsx')

In [8]:
final_full_df

Inclusion Criteria: All Test Set                                          \
                          Threshold    tp    fn     fp     tn    acc    ppv   
0                          ML_0.005  9512     5  66954  24610 0.3376 0.1244   
1                           ML_0.01  9502    15  47366  44198 0.5313 0.1671   
2                           ML_0.02  9433    84  31358  60206 0.6889 0.2313   
3                           ML_0.03  9299   218  24119  67445 0.7592 0.2783   
4                           ML_0.04  9190   327  19763  71801 0.8012 0.3174   
5                           ML_0.05  9073   444  16799  74765 0.8294 0.3507   
6                           ML_0.06  8954   563  14675  76889 0.8492 0.3789   
7                           ML_0.07  8864   653  12953  78611 0.8654 0.4063   
8                           ML_0.09  8666   851  10580  80984 0.8869 0.4503   
9                            ML_0.1  8580   937   9680  81884 0.8950 0.4699   
10                          ML_0.15  8168  1349   6741  84823 0.9200 0.5479   
11                           ML_0.2  7794  1723   5132  86432 0.9322 0.6030   
12                          ML_0.25  7428  2089   4047  87517 0.9393 0.6473   
13                           ML_0.3  7057  2460   3280  88284 0.9432 0.6827   
14                           ML_0.4  6362  3155   2174  89390 0.9473 0.7453   
15                           ML_0.5  5624  3893   1468  90096 0.9470 0.7930   
16                             SIRS  5422  4095  11378  80186 0.8469 0.3227   
17                             MEWS  1534  7983   1096  90468 0.9102 0.5833   

                                                               \
      npv   sens   spec     F1 N_Total_Possible_Sepsis_Alerts   
0  0.9998 0.9995 0.2688 0.2213                          76466   
1  0.9997 0.9984 0.4827 0.2863                          56868   
2  0.9986 0.9912 0.6575 0.3750                          40791   
3  0.9968 0.9771 0.7366 0.4332                          33418   
4  0.9955 0.9656 0.7842 0.4778                          28953   
5  0.9941 0.9533 0.8165 0.5128                          25872   
6  0.9927 0.9408 0.8397 0.5403                          23629   
7  0.9918 0.9314 0.8585 0.5658                          21817   
8  0.9896 0.9106 0.8845 0.6026                          19246   
9  0.9887 0.9015 0.8943 0.6178                          18260   
10 0.9843 0.8583 0.9264 0.6688                          14909   
11 0.9805 0.8190 0.9440 0.6946                          12926   
12 0.9767 0.7805 0.9558 0.7077                          11475   
13 0.9729 0.7415 0.9642 0.7109                          10337   
14 0.9659 0.6685 0.9763 0.7048                           8536   
15 0.9586 0.5909 0.9840 0.6772                           7092   
16 0.9514 0.5697 0.8757 0.4121                          16800   
17 0.9189 0.1612 0.9880 0.2526                           2630   

                                                                      \
   N_Total_Possible_Sepsis_Alerts_Per_Day N_Total_First_Sepsis_Alert   
0                                     209                       6557   
1                                     155                       5390   
2                                     111                       3985   
3                                      91                       3300   
4                                      79                       2926   
5                                      70                       2647   
6                                      64                       2437   
7                                      59                       2276   
8                                      52                       2006   
9                                      50                       1922   
10                                     40                       1572   
11                                     35                       1359   
12                                     31                       1217   
13                                     28